In [ ]:
import os
import requests
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

def export_debt_premium(year: int, month: int, out_dir: str = r"..\data\Debt_premium") -> tuple[pd.DataFrame, str]:
    """
    Descarga, filtra y exporta a CSV los contratos premium con Abo Unique = 1.

    Parámetros
    ----------
    year : int
        Año de la consulta (ejemplo: 2025).
    month : int
        Mes de la consulta en formato numérico (ejemplo: 8 o 12).
    out_dir : str, opcional
        Carpeta donde se guardará el archivo. Por defecto ../data/Debt_premium

    Retorna
    -------
    tuple[pd.DataFrame, str]
        - DataFrame filtrado.
        - Ruta completa del archivo generado.
    """
    # Formatear año y mes
    year_str = str(year)
    month_str = f"{month:02d}"  # asegura siempre 2 dígitos (ej: 08, 12)

    base_url = "https://billing.izoswap.fr/admin/export/suivi_rebill/ALL"
    token = os.getenv("PAYMENTS_TOKEN")
    url = f"{base_url}/{year_str}-{month_str}/{token}/csv"

    # Descargar CSV en memoria
    response = requests.get(url)
    response.raise_for_status()

    df = pd.read_csv(StringIO(response.text), sep=";")

    # Agregar columna UID
    df["UID"] = df["ID Contract"].astype(str).str.extract(r"_(.*)$")

    # Filtro
    mask = (df["Abo Unique"] == 1) & (df["Role"] == "Membre Premium")
    columns = ["UID", "Date of Subscription", "Periodicity"]
    df_filtered = df.loc[mask, columns]

    # Crear carpeta si no existe
    os.makedirs(out_dir, exist_ok=True)

    # Definir nombre del archivo
    file_name = f"Debt_premium_{year_str}_{month_str}.csv"
    file_path = os.path.join(out_dir, file_name)

    # Exportar
    df_filtered.to_csv(file_path, index=False)

    return df_filtered, file_path


In [ ]:
df_premium, ruta = export_debt_premium(2025, 8)
print(df_premium.shape)
print(f"Archivo generado en: {ruta}")

(1193, 3)
Archivo generado en: ..\data\Debt_premium\Debt_premium_2025_08_20250820_200505.csv


In [ ]:
for i in range(1,8):
    df_premium, ruta = export_debt_premium(2025, i)
    print(f"Mes {i:02d} -> {df_premium.shape[0]} filas exportadas en {ruta}")

Mes 01 -> 4458 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_01.csv
Mes 02 -> 4157 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_02.csv


C:\Users\nicol\AppData\Local\Temp\ipykernel_26208\1499641961.py:41: DtypeWarning: Columns (42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(response.text), sep=";")


Mes 03 -> 3218 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_03.csv
Mes 04 -> 3068 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_04.csv
Mes 05 -> 2205 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_05.csv
Mes 06 -> 2222 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_06.csv
Mes 07 -> 1717 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_07.csv
Mes 08 -> 1193 filas exportadas en ..\data\Debt_premium\Debt_premium_2025_08.csv
